In [1]:
from google.cloud import bigquery

import pathlib # __file__ isn't recognized in Jupyter, so we need this to get the root path
path = pathlib.Path.cwd()
PROJ_ROOT = path.parent

accountkeyfile = str(PROJ_ROOT.parent / 'credentials\GoogleServiceAccountKeyFile.json')
client = bigquery.Client.from_service_account_json(accountkeyfile)

#### Accounts Per Device

Device determined by device.advertising_id

In [2]:
query = """
SELECT
    device.advertising_id AS device_id,
    COUNT(DISTINCT user_properties.value.string_value) AS Usernames,
    COUNT(DISTINCT user_pseudo_id) AS Pseudo_IDs
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE 
    ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
GROUP BY
    device.advertising_id
ORDER BY 
    COUNT(DISTINCT user_properties.value.string_value) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,device_id,Usernames,Pseudo_IDs
0,733bdd53-745a-4e95-b32e-9595690aaf52,22,36
1,788dc562-cc05-44c0-bc26-7e24ff3c3768,18,50
2,2f84376a-841a-4d6f-83fc-2d339294e02d,4,4
3,42479d6a-a2e2-4530-be2f-1cc0a77245f0,3,4
4,b3c6b09d-e8b0-413d-a285-bce02fdfb58c,2,2


### Total # of Events by Phone model

In [3]:
query = """
SELECT
    COUNT(device.mobile_os_hardware_model) AS events,
    user_properties.value.string_value,
    device.mobile_os_hardware_model AS model
FROM
    `heycharlie-ada47.analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    event_date > '20181109'
    AND user_properties.value.string_value in ('+vinny', 'Shad Christie', 'Driverseat', 'Blonco', 'Doyle Coleman', 'JoelTOrtega')
GROUP BY
    device.mobile_os_hardware_model,
    user_properties.value.string_value
ORDER BY
    COUNT(device.mobile_os_hardware_model) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,events,string_value,model
0,21243,+vinny,Pixel


### Total # of Events by Username

In [4]:
query = """
SELECT
    COUNT(user_properties.value.string_value) AS events,
    user_properties.value.string_value AS Username
FROM
    `heycharlie-ada47.analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    event_date > '20181109'
    AND user_properties.value.string_value in ('+vinny', 'Shad Christie', 'Driverseat', 'Blonco', 'Doyle Coleman', 'JoelTOrtega')
GROUP BY
    user_properties.value.string_value
ORDER BY
    COUNT(user_properties.value.string_value) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,events,Username
0,21243,+vinny


### # of Notification Events by Username

TODO - Break out notifications by type (positive reinforcement 1x/day, texts, calls)

TODO - Look at responses to notifications

In [5]:
query = """
SELECT
    user_properties.value.string_value as username,
    COUNT(*) AS count
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties,
    UNNEST(event_params) AS event_params
WHERE 
    event_params.key = 'notification_event'
    AND event_date > '20190101'
    AND user_properties.value.string_value in ('Shad Christie', 'Driverseat', 'Blonco', 'Doyle Coleman', 'JoelTOrtega')
GROUP BY
    user_properties.value.string_value
ORDER BY
    COUNT(*) DESC
"""
df = client.query(query).to_dataframe()
df.head(5)

,username,count
0,Shad Christie,516
1,Doyle Coleman,319
2,Blonco,4
3,Driverseat,3
4,JoelTOrtega,1


In [7]:
# string = ', '.join(weekday_dict[list])
# print(string)

In [8]:
query = """
SELECT
    event_name AS event_name,
    COUNT(event_name) AS count,
    user_properties.value.string_value
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties,
    UNNEST(event_params) AS event_params
WHERE 
    event_params.key = 'notification_event'
    AND user_properties.value.string_value in ('Shad Christie', 'Driverseat', 'Blonco', 'Doyle Coleman', 'JoelTOrtega')
GROUP BY
    event_name,
    user_properties.value.string_value
ORDER BY
    event_name
"""
df = client.query(query).to_dataframe()
df.head(50)

,event_name,count,string_value
0,daily_task_notification,2,Shad Christie
1,daily_task_notification,2,Blonco
2,daily_task_notification,2,Doyle Coleman
3,daily_task_notification,1,Driverseat
4,daily_task_notification,1,JoelTOrtega
5,first_week_questionnaire_answer,1,Shad Christie
6,first_week_questionnaire_cancel,1,Blonco
7,first_week_questionnaire_cancel,1,Driverseat
8,first_week_questionnaire_open,1,Shad Christie
9,first_week_questionnaire_open,1,Blonco


### Daily Count of Risky Text Events

In [9]:
query = """
SELECT
    event_date AS Date,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification,
    COUNT(event_name) AS events
FROM 
    (SELECT DISTINCT event_date, event_name, event_server_timestamp_offset
        FROM `analytics_153084895.events_*`)
WHERE
    event_name = "risky_sent_sms_warning_event"
    OR event_name = "risky_sent_sms_warning_notification"
    OR event_name = "risky_sms_event"
    OR event_name = "risky_sms_warning_notification"
GROUP BY
    event_date
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head()

,Date,risky_sent_sms,risky_sent_notification,risky_in_sms,risky_in_notification,events
0,20190418,6,5,3,2,16
1,20190417,3,2,5,5,15
2,20190416,6,5,4,4,19
3,20190415,6,6,5,5,22
4,20190414,9,7,6,5,27


### Daily Risky Events and Notifications, by User

In [10]:
query = """
SELECT
    user_properties.value.string_value AS username,
    SUM(CASE WHEN event_name = "text_message_received" THEN 1 ELSE 0 END) AS in_sms,
    SUM(CASE WHEN event_name = "text_message_sent" THEN 1 ELSE 0 END) AS sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification
FROM 
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_properties.value.string_value in ('Shad Christie', 'Driverseat', 'Blonco', 'Doyle Coleman', 'JoelTOrtega', 'Pinky618 ')
    AND (event_name = "risky_sent_sms_warning_event"
        OR event_name = "risky_sent_sms_warning_notification"
        OR event_name = 'text_message_received'
        OR event_name = 'text_message_sent'
        OR event_name = "risky_sms_event"
        OR event_name = "risky_sms_warning_notification")
GROUP BY
    user_properties.value.string_value
"""
df = client.query(query).to_dataframe()
df.head(100)

,username,in_sms,sent_sms,risky_sent_sms,risky_sent_notification,risky_in_sms,risky_in_notification
0,Doyle Coleman,64,5,0,0,0,0
1,Shad Christie,162,77,0,0,0,0
2,Driverseat,2,0,0,0,0,0
3,Pinky618,226,63,16,3,25,4
4,JoelTOrtega,29,31,0,0,0,0


In [11]:
query = """
SELECT
    event_date AS Date,
    user_properties.value.string_value AS username,
    SUM(CASE WHEN event_name = "text_message_received" THEN 1 ELSE 0 END) AS in_sms,
    SUM(CASE WHEN event_name = "text_message_sent" THEN 1 ELSE 0 END) AS sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification
FROM 
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_properties.value.string_value in ('Shad Christie', 'Driverseat', 'Blonco', 'Doyle Coleman', 'JoelTOrtega', 'Pinky618 ')
    AND (event_name = "risky_sent_sms_warning_event"
        OR event_name = "risky_sent_sms_warning_notification"
        OR event_name = 'text_message_received'
        OR event_name = 'text_message_sent'
        OR event_name = "risky_sms_event"
        OR event_name = "risky_sms_warning_notification")
GROUP BY
    event_date, user_properties.value.string_value
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head(100)

,Date,username,in_sms,sent_sms,risky_sent_sms,risky_sent_notification,risky_in_sms,risky_in_notification
0,20190419,Driverseat,2,0,0,0,0,0
1,20190417,Pinky618,4,0,0,0,0,0
2,20190416,Pinky618,6,0,0,0,0,0
3,20190415,Doyle Coleman,3,0,0,0,0,0
4,20190415,Shad Christie,0,3,0,0,0,0
5,20190413,Pinky618,2,0,0,0,0,0
6,20190411,Shad Christie,9,7,0,0,0,0
7,20190411,Doyle Coleman,0,1,0,0,0,0
8,20190410,Pinky618,26,0,10,1,18,2
9,20190409,Pinky618,6,0,0,0,0,0


### Days of App Use

In [12]:
query = """
SELECT
    event_date AS Date,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification,
    user_properties.value.string_value AS username,
    COUNT(event_name) AS total_risly_sms_events
FROM 
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_properties.value.string_value <> "null"
    AND (event_name = "risky_sent_sms_warning_event"
        OR event_name = "risky_sent_sms_warning_notification"
        OR event_name = "risky_sms_event"
        OR event_name = "risky_sms_warning_notification")
GROUP BY
    event_date, user_properties.value.string_value
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,risky_sent_sms,risky_sent_notification,risky_in_sms,risky_in_notification,username,total_risly_sms_events
0,20190418,4,3,8,4,Charlie,19
1,20190418,4,2,9,0,Jake,15
2,20190417,1,0,7,3,Jake,11
3,20190417,2,2,8,2,Charlie,14
4,20190416,12,3,27,4,Jake,46
5,20190416,4,4,3,1,Charlie,12
6,20190415,1,1,0,0,mckay68,2
7,20190415,14,4,22,5,Jake,45
8,20190415,1,1,2,1,Charlie,5
9,20190414,11,8,0,0,mckay68,19


### Days of App Use

In [13]:
query = """
SELECT
    MIN(event_date) AS Start_Date,
    MAX(event_date) AS Max_Date,
    ROUND((MAX(CAST(event_timestamp AS INT64)) - MIN(CAST(event_timestamp AS INT64))) / (86400 * 1000000),1) AS Installed_Days,
    user_properties.value.string_value AS Username,
    MIN(user_first_touch_timestamp) = MIN(event_timestamp) AS first_touch_equal_first_event,
    MIN(user_first_touch_timestamp) AS first_touch_timestamp,
    MIN(event_timestamp) AS min_event_timestamp,
    MAX(event_timestamp) AS max_timestamp
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_first_touch_timestamp > 20181110
    AND user_properties.value.string_value in ('Shad Christie', 'Driverseat', 'Blonco', 'Doyle Coleman', 'JoelTOrtega')
GROUP BY
    user_properties.value.string_value
ORDER BY
    (MAX(CAST(event_timestamp AS INT64)) - MIN(CAST(event_timestamp AS INT64))) DESC
"""
df = client.query(query).to_dataframe()
df.head(5)

,Start_Date,Max_Date,Installed_Days,Username,first_touch_equal_first_event,first_touch_timestamp,min_event_timestamp,max_timestamp
0,20190311,20190416,35.6,Shad Christie,False,1552331570671000,1552331642821000,1555403372879284
1,20190319,20190419,31.2,Doyle Coleman,False,1553005169571000,1553005361651000,1555700677040753
2,20190319,20190419,31.0,Driverseat,False,1553005074463000,1553005140856000,1555687686064901
3,20190319,20190418,30.3,Blonco,False,1553004947390000,1553005035909000,1555621201439000
4,20190319,20190319,0.4,JoelTOrtega,False,1553004788248000,1553004891710000,1553041245108001


### Installs

In [14]:
query = """
SELECT
    MIN(event_date) AS Date,
    MIN(event_timestamp) AS min_event_timestamp,
    MIN(user_first_touch_timestamp) AS Install,
    user_properties.value.string_value AS Username,
    geo.region AS state
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
GROUP BY
    user_properties.value.string_value,
    geo.region
ORDER BY 
    MIN(event_date) DESC
"""
df = client.query(query).to_dataframe()
df.head(40)

,Date,min_event_timestamp,Install,Username,state
0,20190419,1555693331944302,1555693282173000,kylecav,Ohio
1,20190418,1555610344198000,1555610303799000,+Vinny,Massachusetts
2,20190418,1555607402239000,1555381101355000,test3,Florida
3,20190417,1555495204720000,1554044615309000,Charlie,Rhode Island
4,20190417,1555558394792000,1549071668527000,dapnac3,Ohio
5,20190416,1555434903607000,1554044615309000,Charlie,New York
6,20190416,1555435468456000,1555435417001000,janetverwolf,Washington
7,20190416,1555435461456000,1555435369880000,DavidT,New York
8,20190415,1555366383960259,1554044615309000,Charlie,Massachusetts
9,20190414,1555258996339494,1545615800164000,Twhite,South Carolina


### Event Count For Date Range

TABLE_DATE_RANGE is legacy SQL and seems to only work in the BigQuery web portal

In [15]:
query = """
SELECT
    COUNT(event_name) AS NotificationEvents,
    event_name,
    event_params.value.string_value AS string_value
FROM
    `analytics_153084895.*`,
    UNNEST(event_params) AS event_params
WHERE
    _TABLE_SUFFIX IN ('events_20190122', 'events_20191231')
GROUP BY
    event_name, string_value
ORDER BY
    NotificationEvents DESC
"""

# This is the legacy SQL
# query = """ 
# SELECT
#     COUNT(event_name) AS NotificationEvents,
#     event_name,
#     event_params.value.string_value AS string_value
# FROM
#     TABLE_DATE_RANGE( analytics_153084895.events_,
#         TIMESTAMP('2018-01-01'),
#         TIMESTAMP('2018-12-31'))
# GROUP BY
#     event_name, string_value
# ORDER BY
#     NotificationEvents DESC
# """

df = client.query(query).to_dataframe()
df.head(10)

,NotificationEvents,event_name,string_value
0,449,text_message_received,None
1,449,text_message_received,app
2,448,reviver_scimitar_already_running,app
3,400,text_message_received,sms
4,252,data_sync_friend_event_3_tries,app
5,223,text_message_received,SMSMonitoringService
6,223,text_message_received,received
7,194,text_message_sent,app
8,194,text_message_sent,sms
9,177,text_message_received,BroadcastReceiver


### First Week Feedback

In [16]:
query = """
SELECT
    event_date AS date,
    event_timestamp AS timestamp,
    user_properties.value.string_value as username,
    (SELECT event_params.value.string_value
    FROM UNNEST(event_params) AS event_params
    WHERE event_params.key = 'face') AS face,
    (SELECT event_params.value.string_value
    FROM UNNEST(event_params) AS event_params
    WHERE event_params.key = 'feedback') AS feedback
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE 
    event_name = 'first_week_questionnaire_answer'
    AND user_properties.value.string_value IS NOT NULL
ORDER BY
    event_timestamp DESC
"""
df = client.query(query).to_dataframe()
df

,date,timestamp,username,face,feedback
0,20190418,1555639409202003,vinnov10,nuetral,Lol cats
1,20190328,1553813094042043,test3,sad,
2,20190322,1553296256234003,+vinny,nuetral,Test\n
3,20190320,1553119114338010,fred13,happy,
4,20190318,1552954589875020,test3,sad,
5,20190318,1552953661172003,Shad Christie,happy,None
6,20190314,1552600269860004,+vinny,nuetral,Test
7,20190313,1552514434012006,gmbs,happy,
8,20190219,1550631390305006,amullins,happy,Alsome
9,20190219,1550616825140006,+vinny,nuetral,Dhd


### Onboarding Events

In [17]:
query = """
SELECT
    t1.event_date AS Date,
    t1.event_name AS event_name,
    t1.user_pseudo_id AS user_pseudo_id,
    t2.username as Username,
    t1.event_timestamp AS timestamp,
    t1.event_previous_timestamp AS previous_timestamp,
    t1.user_first_touch_timestamp AS first_touch_timestamp,
    t2.done_timestamp AS done_timestamp,
    (SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') AS screen,
    (SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_previous_class') AS prev_screen
FROM
    `analytics_153084895.events_*` as t1,
    UNNEST(t1.user_properties) AS user_properties
JOIN(
    SELECT
        MIN(event_timestamp) AS done_timestamp,
        user_properties.value.string_value AS username
    FROM
        `analytics_153084895.events_*`,
         UNNEST(user_properties) AS user_properties
    WHERE 
        ((SELECT event_params.value.string_value
            FROM UNNEST(event_params) AS event_params
            WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
    GROUP BY
        user_properties.value.string_value) AS t2
ON t2.username = user_properties.value.string_value
WHERE
    t1.event_timestamp < t2.done_timestamp
    AND t1.event_name = 'screen_view'
    AND t2.username != 'null'
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,event_name,user_pseudo_id,Username,timestamp,previous_timestamp,first_touch_timestamp,done_timestamp,screen,prev_screen
0,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688800078011,1546688792534011,1546688739845000,1546689262192075,PermissionsActivity,DexterActivity
1,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688939033038,1546688939015038,1546688739845000,1546689262192075,PlacesActivity,PlacesActivity
2,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688939477039,1546688939033039,1546688739845000,1546689262192075,RelationshipsMessageActivity,PlacesActivity
3,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688987361045,1546688942094045,1546688739845000,1546689262192075,RelationshipsMessageActivity,PlacesActivity
4,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688785880001,1546688742976001,1546688739845000,1546689262192075,WelcomeActivity,SignupActivity
5,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688785927002,1546688785880002,1546688739845000,1546689262192075,WelcomeActivity,WelcomeActivity
6,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688887566022,1546688887556022,1546688739845000,1546689262192075,WelcomeActivity,WelcomeActivity
7,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688787913004,1546688785927004,1546688739845000,1546689262192075,LegalAbstractActivity,WelcomeActivity
8,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688888561023,1546688887566023,1546688739845000,1546689262192075,LegalAbstractActivity,WelcomeActivity
9,20190105,screen_view,c9f46ced65178ff00295313bbbec0ce3,kaltmann19,1546688893941032,1546688893923032,1546688739845000,1546689262192075,AboutYouActivity,AboutYouActivity


### Onboarding Screen View Timing

In [18]:
query = """
SELECT
    t1.event_date AS Date,
    t1.event_name AS event_name,
    t1.user_pseudo_id AS user_pseudo_id,
    t2.username as Username,
    t1.event_timestamp AS timestamp,
    t1.event_previous_timestamp AS previous_timestamp,
    t1.user_first_touch_timestamp AS first_touch_timestamp,
    t2.done_timestamp AS done_timestamp,
    (SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_previous_class') AS Screen
FROM
    `analytics_153084895.events_*` as t1,
    UNNEST(t1.user_properties) AS user_properties
JOIN(
    SELECT
        MIN(event_timestamp) AS done_timestamp,
        user_properties.value.string_value AS username
    FROM
        `analytics_153084895.events_*`,
        UNNEST(user_properties) AS user_properties
    WHERE 
        ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
        GROUP BY
            user_properties.value.string_value) AS t2
ON t2.username = user_properties.value.string_value
WHERE
    t1.event_timestamp < t2.done_timestamp
    AND t1.event_name = 'screen_view'
    AND t2.username != 'null'
    AND ((SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') != 
            (SELECT event_params.value.string_value
            FROM UNNEST(t1.event_params) AS event_params
            WHERE event_params.key = 'firebase_previous_class'))
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,event_name,user_pseudo_id,Username,timestamp,previous_timestamp,first_touch_timestamp,done_timestamp,Screen
0,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113170162002,1553112815444002,1553112810129000,1553114289899047,SignupActivity
1,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113173565005,1553113170179005,1553112810129000,1553114289899047,WelcomeActivity
2,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113176880008,1553113173582008,1553112810129000,1553114289899047,LegalAbstractActivity
3,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113204482011,1553113176899011,1553112810129000,1553114289899047,PermissionsActivity
4,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113258175014,1553113221679014,1553112810129000,1553114289899047,DexterActivity
5,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113265592018,1553113258175018,1553112810129000,1553114289899047,PermissionsActivity
6,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113295145022,1553113265606022,1553112810129000,1553114289899047,NotificationsPermissionActivity
7,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113505346025,1553113295202025,1553112810129000,1553114289899047,AboutYouActivity
8,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113575325028,1553113505356028,1553112810129000,1553114289899047,PlacesMessageActivity
9,20190320,screen_view,dd9f6c0a576c2c9164b58b37ad984b2d,Pinky618,1553113641511031,1553113575340031,1553112810129000,1553114289899047,PlacesActivity


### Total Onboarding Time

In [19]:
query = """
SELECT
    MIN(event_date) AS Date,
    user_properties.value.string_value AS Username,
    MIN(event_timestamp) AS end_timestamp,
    MIN(user_first_touch_timestamp) AS first_touch_timestamp
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE 
    event_name = 'screen_view'
    AND ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
GROUP BY
    user_properties.value.string_value,
    (SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class')
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,Username,end_timestamp,first_touch_timestamp
0,20190320,Pinky618,1553114289899047,1553112810129000
1,20181217,test17,1545101139474059,1542476516676000
2,20190103,mattiet,1546538750331019,1546537939532000
3,20180906,test6,1536290799526028,1536287952446000
4,20190110,hbickers,1547138320091091,1547137724581000
5,20181029,test26,1540871324531021,1537833795897000
6,20181205,atuck,1544026782549050,1544026297038000
7,20180807,vinnyvinny,1533702674562085,1533701855969000
8,20181120,shanila,1542757054365078,1542745129834000
9,20190202,Stevehburns,1549139041821042,1549138749264000


### Post-Onboarding Screen View Timing

In [20]:
query = """
SELECT
    t1.event_date AS Date,
    t1.event_name AS event_name,
    t2.username as Username,
    t1.event_timestamp AS timestamp,
    t1.event_previous_timestamp AS previous_timestamp,
    t1.user_first_touch_timestamp AS first_touch_timestamp,
    (SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_previous_class') AS Screen
FROM
    `analytics_153084895.events_*` as t1,
    UNNEST(t1.user_properties) AS user_properties
JOIN(
    SELECT
        MIN(event_timestamp) AS done_timestamp,
        user_properties.value.string_value AS username
    FROM
        `analytics_153084895.events_*`,
        UNNEST(user_properties) AS user_properties
    WHERE 
        ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
    GROUP BY
        user_properties.value.string_value) AS t2
ON t2.username = user_properties.value.string_value
WHERE
    t1.event_timestamp > t2.done_timestamp
    AND t1.event_name = 'screen_view'
    AND t2.username = 'Shad Christie'
    AND ((SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') != 
            (SELECT event_params.value.string_value
            FROM UNNEST(t1.event_params) AS event_params
            WHERE event_params.key = 'firebase_previous_class'))
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,event_name,Username,timestamp,previous_timestamp,first_touch_timestamp,Screen
0,20190319,screen_view,Shad Christie,1553013666324011,1553013666126011,1552331570671000,SigninSignupActivity
1,20190319,screen_view,Shad Christie,1553014783652001,1553013666339001,1552331570671000,HomeActivity
2,20190319,screen_view,Shad Christie,1553014783911007,1553014783652007,1552331570671000,SigninSignupActivity
3,20190319,screen_view,Shad Christie,1553014787671015,1553014783944015,1552331570671000,HomeActivity
4,20190319,screen_view,Shad Christie,1553014794752020,1553014787743020,1552331570671000,AboutYouActivity
5,20190319,screen_view,Shad Christie,1553014799612024,1553014794777024,1552331570671000,HomeActivity
6,20190319,screen_view,Shad Christie,1553014802344001,1553014799711001,1552331570671000,SettingsActivity
7,20190319,screen_view,Shad Christie,1553014807760006,1553014802413006,1552331570671000,HomeActivity
8,20190319,screen_view,Shad Christie,1553014810700009,1553014807790009,1552331570671000,SettingsActivity
9,20190319,screen_view,Shad Christie,1553014822820016,1553014810853016,1552331570671000,HomeActivity


### Screen View Timing Data

In [21]:
query = """
SELECT
    (SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_previous_class') AS Screen,
    event_date AS Date,
    user_properties.value.string_value AS Username,
    event_timestamp AS end_timestamp,
    event_previous_timestamp AS start_timestamp
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE 
    event_name = 'screen_view'
    AND ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') != 
            (SELECT event_params.value.string_value
            FROM UNNEST(event_params) AS event_params
            WHERE event_params.key = 'firebase_previous_class'))
ORDER BY
    event_timestamp
"""
df = client.query(query).to_dataframe()
df.head(10)

,Screen,Date,Username,end_timestamp,start_timestamp
0,HomeActivity,20180711,None,1531358521175000,1531257346796000
1,SigninSignupActivity,20180711,None,1531358531292002,1531358521175002
2,SignupActivity,20180711,None,1531358536305003,1531358531292003
3,SigninSignupActivity,20180711,None,1531358536478004,1531358536305004
4,SignupActivity,20180711,None,1531358937284006,1531358536478006
5,HomeActivity,20180712,None,1531376260465000,1531259406844000
6,SigninSignupActivity,20180712,None,1531376262537002,1531376260465002
7,SigninActivity,20180712,None,1531376312750005,1531376262537005
8,HomeActivity,20180712,None,1531376320627007,1531376312750007
9,SettingsActivity,20180712,None,1531376325091009,1531376320627009


### SMS vs MMS

In [22]:
query = """
SELECT
    type as type,
    COUNT(username) AS events
FROM
    (SELECT
        (SELECT event_params.value.string_value
            FROM UNNEST(event_params) AS event_params
            WHERE event_params.key = 'type') AS type,
        event_date AS date,
        user_properties.value.string_value AS username,
        event_timestamp AS timestamp,
        event_previous_timestamp AS previous_timestamp
    FROM
        `analytics_153084895.events_*`,
         UNNEST(user_properties) AS user_properties
    WHERE 
        event_name = 'text_message_received' OR event_name = "risky_sent_sms_warning_event")
GROUP BY
    type
"""
df = client.query(query).to_dataframe()
df.head(10)

,type,events
0,sms,111258
1,mms,16263


### Time Between Texts

In [23]:
query = """
SELECT
    t1.event_date AS date,
    t1.event_name AS event_name,
    LAG(t1.event_timestamp) OVER (PARTITION BY username ORDER BY t1.event_timestamp) AS previous_timestamp,
    t1.event_timestamp AS current_timestamp,
    t1.event_timestamp - LAG(t1.event_timestamp) OVER (PARTITION BY username ORDER BY t1.event_timestamp) AS time_delta,
    username AS username
FROM 
    (SELECT MAX(event_date) AS event_date, MAX(event_timestamp) AS event_timestamp, event_name, user_properties.value.string_value AS username
        FROM `analytics_153084895.events_*`,
        UNNEST(user_properties) AS user_properties
        GROUP BY event_name, event_server_timestamp_offset, username
        ) AS t1
WHERE
    username = 'Shad Christie' AND 
    (t1.event_name = "risky_sent_sms_warning_event" OR event_name = "risky_sms_event")
LIMIT 100
"""
df = client.query(query).to_dataframe()
df.head(10)

,date,event_name,previous_timestamp,current_timestamp,time_delta,username


### Unique Device Models

In [24]:
query = """
SELECT
    COUNT(device.mobile_os_hardware_model),
    device.mobile_os_hardware_model AS model
FROM
    `heycharlie-ada47.analytics_153084895.events_*`
WHERE
    event_date > '20181109'
GROUP BY
    device.mobile_os_hardware_model
ORDER BY
    COUNT(device.mobile_os_hardware_model) DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

,f0_,model
0,2066645,SM-G930P
1,167742,LG-LS997
2,51772,SM-N920V
3,50621,Pixel 3 XL
4,40799,SAMSUNG-SM-G891A
5,38946,Pixel XL
6,34952,A577VL
7,34292,SM-G950U
8,27731,Pixel
9,26149,SM-J737T1


### Weekly Questionnaire Instances

TODO It looks like the event_name may have changed...

In [25]:
query = """
SELECT
    event_date AS Date,
    COUNT(event_name) AS event
FROM
    `analytics_153084895.events_*`,
    UNNEST(event_params) AS event_params
WHERE
    event_name = "weekly_questionnaire_notification" AND key = "notification_event"
GROUP BY
    event_date
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,event
0,20180911,1
1,20180906,5
2,20180905,1
3,20180828,3
4,20180827,1
5,20180820,1
6,20180814,1
7,20180813,2
8,20180811,1
9,20180808,1
